In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
my_df = pd.read_csv('drive/MyDrive/datasets/pre_data_dl_aug2.csv')
#my_df = pd.read_csv(csv,index_col=0)
my_df.head()

,Unnamed: 0,Combined Description Cleaned,Assignment group,LabelEncodings
0,0,login issue user manager name checked the name...,GRP_0,0
1,1,outlook received from hello team my are not in...,GRP_0,0
2,2,cannot log in to received from hi i cannot on ...,GRP_0,0
3,3,unable to access tool page,GRP_0,0
4,4,error,GRP_0,0


In [5]:
df2 = my_df
my_df['Assignment group by number'] = my_df['Assignment group'].str[4:]
my_df['Assignment group by number'] = my_df['Assignment group by number'].astype(int)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31221 entries, 0 to 31220
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Unnamed: 0                    31221 non-null  int64 
 1   Combined Description Cleaned  31221 non-null  object
 2   Assignment group              31221 non-null  object
 3   LabelEncodings                31221 non-null  int64 
 4   Assignment group by number    31221 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.2+ MB


In [6]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31221 entries, 0 to 31220
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Unnamed: 0                    31221 non-null  int64 
 1   Combined Description Cleaned  31221 non-null  object
 2   Assignment group              31221 non-null  object
 3   LabelEncodings                31221 non-null  int64 
 4   Assignment group by number    31221 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.2+ MB


In [7]:
x = my_df['Combined Description Cleaned']
y = my_df['Assignment group by number']

In [8]:
import tensorflow as tf
y = tf.keras.utils.to_categorical(y)

In [9]:
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.20, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [10]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

In [11]:
def labelize_tickets_ug(tickets,label):
    result = []
    prefix = label
    for i, t in zip(tickets.index, tickets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [12]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tickets_ug(all_x, 'all')

In [13]:
cores = multiprocessing.cpu_count()
model_ug_cbow = Word2Vec(sg=0, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 31221/31221 [00:00<00:00, 2219159.20it/s]


In [14]:
%%time
for epoch in range(30):
    model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_cbow.alpha -= 0.002
    model_ug_cbow.min_alpha = model_ug_cbow.alpha

100%|██████████| 31221/31221 [00:00<00:00, 2672892.82it/s]


CPU times: user 38 s, sys: 389 ms, total: 38.4 s
Wall time: 12 s


In [15]:
model_ug_sg = Word2Vec(sg=1, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 31221/31221 [00:00<00:00, 2148664.62it/s]


In [16]:
%%time
for epoch in range(30):
    model_ug_sg.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_sg.alpha -= 0.002
    model_ug_sg.min_alpha = model_ug_sg.alpha

100%|██████████| 31221/31221 [00:00<00:00, 2595903.76it/s]


CPU times: user 1min 16s, sys: 324 ms, total: 1min 16s
Wall time: 20.7 s


In [17]:
model_ug_cbow.save('drive\MyDrive\grams\w2v_model_ug_cbow.word2vec')
model_ug_sg.save('drive\MyDrive\grams\w2v_model_ug_sg.word2vec')

# Preparation for Convolutional Neural Network

In the last post, I have aggregated the word vectors of each word in a ticket, either summation or calculating mean to get one vector representation of each ticket. However, in order to feed to a CNN, we have to not only feed each word vector to the model, but also in a sequence which matches the original ticket desc.

For example, let's say we have a sentence as below.

"I want password reset"

And let's assume that we have a 2-dimensional vector representation of each word as follows:

I: [0.3, 0.5]
want: [1.2, 0.8]
password: [0.4, 1.3]

With the above sentence, the dimension of the vector we have for the whole sentence is 3 X 2 (3: number of words, 2: number of vector dimension).

But there is one more thing we need to consider. A neural network model will expect all the data to have the same dimension, but in case of different sentences, they will have different lengths. This can be handled with padding.

The first sentence had 3X2 dimension vectors, but the second sentence has 4X2 dimension vector. Our neural network won't accept these as inputs. By padding the inputs, we decide the maximum length of words in a sentence, then zero pads the rest, if the input length is shorter than the designated length. In the case where it exceeds the maximum length, then it will also truncate either from the beginning or from the end. For example, let's say we decide our maximum length to be 5.

Then by padding, the first sentence will have 2 more 2-dimensional vectors of all zeros at the start or the end (you can decide this by passing an argument), and the second sentence will have 1 more 2-dimensional vector of zeros at the beginning or the end. Now we have 2 same dimensional (5X2) vectors for each sentence, and we can finally feed this to a model.

Let's first load the Word2Vec models to extract word vectors from. I have saved the Word2Vec models I trained in the previous post, and can easily be loaded with "KeyedVectors" function in Gensim. I have two different Word2Vec models, one with CBOW (Continuous Bag Of Words) model, and the other with a skip-gram model. I won't go into detail of how CBOW and skip-gram differs, but you can refer to my previous post if you want to know a bit more in detail.

In [18]:
from gensim.models import KeyedVectors
model_ug_cbow = KeyedVectors.load('drive\MyDrive\grams\w2v_model_ug_cbow.word2vec')
model_ug_sg = KeyedVectors.load('drive\MyDrive\grams\w2v_model_ug_sg.word2vec')

In [19]:
len(model_ug_cbow.wv.vocab.keys())

3548

By running below code block, I am constructing a sort of dictionary I can extract the word vectors from. Since I have two different Word2Vec models, below "embedding_index" will have concatenated vectors of the two models. For each model, I have 100 dimension vector representation of the words, and by concatenating each word will have 200 dimension vector representation.

In [20]:
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = np.append(model_ug_cbow.wv[w],model_ug_sg.wv[w])
print('Found %s word vectors.' % len(embeddings_index))

Found 3548 word vectors.


Now we have our reference to word vectors ready, but we still haven't prepared data to be in the format I have explained at the start of the post. Keras' 'Tokenizer' will split each word in a sentence,  then we can call 'texts_to_sequences' method to get the sequential representation of each sentence. We also need to pass 'num_words' which is a number of vocabularies you want to use, and this will be applied when you call 'texts_to_sequences' method. This might be a bit counter-intuitive since if you check the length of all the word index, it will not be the number of words you defined, but the actual screening process happens when you call 'texts_to_sequences' method.

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

In [22]:
len(tokenizer.word_index)

4256

Below are the first five entries of the original train data.

In [23]:
for x in x_train[:5]:
    print( x)

metal cutting technology company hi team i have the attached quote however it is not up showing correctly see attached can you please fix this as soon possible
office is not defined for area when we new order got communicate that incomplete sale try enter right i not add the print screen in attached
unable to access engineering tool received from dear sir following error message shown request support resolve
unable to share screen language explorer customer number telephone summary am my on
trigger making need rate th


And the same data prepared as sequential dats is as below.

In [24]:
sequences[:5]

[[2048,
  1529,
  914,
  35,
  84,
  71,
  16,
  20,
  2,
  89,
  466,
  340,
  13,
  5,
  7,
  56,
  236,
  317,
  64,
  89,
  31,
  23,
  6,
  264,
  17,
  39,
  561,
  171],
 [197,
  5,
  7,
  932,
  10,
  352,
  33,
  40,
  83,
  101,
  274,
  1798,
  34,
  1321,
  1663,
  276,
  196,
  484,
  16,
  7,
  244,
  2,
  169,
  140,
  4,
  89],
 [60, 1, 36, 120, 22, 12, 3, 145, 418, 138, 29, 57, 476, 135, 219, 260],
 [60, 1, 737, 140, 215, 240, 102, 105, 245, 308, 38, 50, 9],
 [1710, 478, 42, 915, 322]]

Each word is represented as a number, and we can see that the number of words in each sentence is matching the length of numbers in the "sequences". We can later make connections of which word each number represents. But we still didn't pad our data, so each sentence has varying length. Let's deal with this.

In [25]:
length = []
for x in x_train:
    length.append(len(x.split()))

In [26]:
max(length)

191

The maximum number of words in a sentence within the training data is 191. Let's decide the maximum length to be a bit longer than this, let's say 200.

In [27]:
x_train_seq = pad_sequences(sequences, maxlen=200)
print('Shape of data tensor:', x_train_seq.shape)

Shape of data tensor: (24976, 200)


In [28]:
x_train_seq[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

As you can see from the padded sequences, all the data now transformed to have the same length of 45, and by default, Keras zero-pads at the beginning, if a sentence length is shorter than the maximum length. If you want to know more in detail, please check the Keras documentation on sequence preprocessing. https://keras.io/preprocessing/sequence/

In [29]:
sequences_val = tokenizer.texts_to_sequences(x_validation)
x_val_seq = pad_sequences(sequences_val, maxlen=200)

There's still one more thing left to do before we can feed the sequential text data to a model. When we transformed a sentence into a sequence, each word is represented by an integer number. Actually, these numbers are where each word is stored in the tokenizer's word index. Keeping this in mind, let's build a matrix of these word vectors, but this time we will use the word index number so that our model can refer to the corresponding vector when fed with integer sequence.

Below, I am defining the number of words to be 100,000. This means I will only care about 100,000 most frequent words in the training set. If I don't limit the number of words, the total number of vocabulary will be more than 200,000.

In [30]:
num_words = 100000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

As a sanity check, if the embedding matrix has been generated properly. In the above, when I saw the first five entries of the training set, the first entry was "hate you", and the sequential representation of this was [137, 6]. Let's see if 6th embedding matrix is as same as vectors for the word 'you'.

In [36]:
embeddings_index.get('network')

array([-1.4111273e-01,  2.9155019e-01, -1.1284587e-01,  2.2359337e-01,
        6.0058411e-02, -4.9900252e-02,  8.5473888e-02, -7.8683376e-01,
        8.6925071e-01,  2.1847136e+00, -9.3664986e-01,  7.3335814e-01,
       -6.8616986e-02, -8.7594610e-01,  1.1092713e-01, -1.2582666e+00,
        1.2278801e+00,  1.2878528e+00, -9.8261803e-01, -7.2262418e-01,
        1.1262896e+00,  9.7218889e-01, -8.3947498e-01, -6.3291377e-01,
       -4.9763948e-01,  5.3166255e-02, -9.0042315e-03, -2.1348071e+00,
       -1.5790586e-01,  9.1814911e-01, -2.8721866e-01, -1.0304505e+00,
       -3.6228693e-01, -3.4313101e-01,  8.9838028e-01, -3.1134722e-01,
        8.2395124e-01, -3.6602399e-01,  8.8751358e-01, -6.9557905e-02,
       -2.3045862e-01, -1.9827213e+00,  5.2359575e-01, -1.7394167e+00,
        3.3287084e-01,  3.7697442e-02, -3.7102136e-01,  1.8132128e+00,
       -1.3087019e-01, -1.1318239e+00,  7.6131320e-01,  1.1042281e+00,
        2.3107922e+00, -8.0040866e-01,  9.7324580e-02, -6.1856121e-01,
      

In [35]:
np.array_equal(embedding_matrix[6] ,embeddings_index.get('network'))

False

Now we are ready with the data preparation. Before we jump into CNN, I would like to test one more thing (sorry for the delay). When we feed this sequential vector representation of data, we will use Embedding layer in Keras. With Embedding layer, I can either pass pre-defined embedding, which I prepared as 'embedding_matrix' above, or Embedding layer itself can learn word embeddings as the whole model trains. And another possibility is we can still feed the pre-defined embedding but make it trainable so that it will update the values of vectors as the model trains.

In order to check which method performs better, I defined a simple shallow neural network one hidden layer. For this model structure, I will not try to refine models by tweaking parameters, since the main purpose of this post is to implement CNN.

https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

In [37]:
seed = 7

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [38]:
model_ptw2v = Sequential()
e = Embedding(100000, 200, weights=[embedding_matrix], input_length=200, trainable=False)
model_ptw2v.add(e)
model_ptw2v.add(Flatten())
model_ptw2v.add(Dense(256, activation='relu'))
model_ptw2v.add(Dense(74, activation='softmax'))
model_ptw2v.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_ptw2v.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

Epoch 1/5
781/781 - 4s - loss: 0.9522 - accuracy: 0.7717 - val_loss: 0.3015 - val_accuracy: 0.9074
Epoch 2/5
781/781 - 3s - loss: 0.2124 - accuracy: 0.9400 - val_loss: 0.2621 - val_accuracy: 0.9190
Epoch 3/5
781/781 - 3s - loss: 0.1496 - accuracy: 0.9534 - val_loss: 0.2496 - val_accuracy: 0.9270
Epoch 4/5
781/781 - 3s - loss: 0.1431 - accuracy: 0.9563 - val_loss: 0.2521 - val_accuracy: 0.9183
Epoch 5/5
781/781 - 3s - loss: 0.1694 - accuracy: 0.9508 - val_loss: 0.6520 - val_accuracy: 0.8725


In [39]:
model_ptw2v = Sequential()
e = Embedding(100000, 200, input_length=200)
model_ptw2v.add(e)
model_ptw2v.add(Flatten())
model_ptw2v.add(Dense(256, activation='relu'))
model_ptw2v.add(Dense(74, activation='softmax'))
model_ptw2v.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_ptw2v.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

Epoch 1/5
781/781 - 92s - loss: 1.7356 - accuracy: 0.5766 - val_loss: 0.5821 - val_accuracy: 0.8523
Epoch 2/5
781/781 - 91s - loss: 0.3180 - accuracy: 0.9137 - val_loss: 0.2827 - val_accuracy: 0.9215
Epoch 3/5
781/781 - 91s - loss: 0.1713 - accuracy: 0.9487 - val_loss: 0.2607 - val_accuracy: 0.9209
Epoch 4/5
781/781 - 91s - loss: 0.1359 - accuracy: 0.9572 - val_loss: 0.2610 - val_accuracy: 0.9209
Epoch 5/5
781/781 - 91s - loss: 0.1185 - accuracy: 0.9600 - val_loss: 0.2448 - val_accuracy: 0.9244


In [40]:
model_ptw2v = Sequential()
e = Embedding(100000, 200, weights=[embedding_matrix], input_length=200, trainable=True)
model_ptw2v.add(e)
model_ptw2v.add(Flatten())
model_ptw2v.add(Dense(256, activation='relu'))
model_ptw2v.add(Dense(74, activation='softmax'))
model_ptw2v.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_ptw2v.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

Epoch 1/5
781/781 - 90s - loss: 0.9620 - accuracy: 0.7673 - val_loss: 0.3163 - val_accuracy: 0.9049
Epoch 2/5
781/781 - 88s - loss: 0.2068 - accuracy: 0.9395 - val_loss: 0.2504 - val_accuracy: 0.9196
Epoch 3/5
781/781 - 85s - loss: 0.1508 - accuracy: 0.9528 - val_loss: 0.2462 - val_accuracy: 0.9215
Epoch 4/5
781/781 - 85s - loss: 0.1387 - accuracy: 0.9578 - val_loss: 0.2318 - val_accuracy: 0.9308
Epoch 5/5
781/781 - 85s - loss: 0.1252 - accuracy: 0.9591 - val_loss: 0.3202 - val_accuracy: 0.9145


As a result, the best validation accuracy is from the third method (fine-tune pre-trained Word2Vec) at 82.22%. The best training accuracy is the second method (learn word embedding from scratch) at 90.52%. Using pre-trained Word2Vec without updating its vector values showed the lowest accuracy both in training and validation. However, what's interesting is that in terms of training set accuracy, fine-tuning pre-trained word vectors couldn't outperform the word embeddings learned from scratch through the embedding layer. Before I tried the above three methods, my first guess was that if I fine-tune the pre-trained word vectors, it would give me the best training accuracy.

Feeding pre-trained word vectors for an embedding layer to update is like providing the first initialisation guideline to the embedding layer so that it can learn more efficiently the task-specific word vectors. But the result is somewhat counterintuitive, and in this case, it turns out that it is better to force the embedding layer to learn from scratch.

But premature generalization is a dangerous step to take. For this reason, I will compare three methods again in the context of CNN.

# Convolutional Neural Network

In [41]:
from keras.layers import Conv1D, GlobalMaxPooling1D

In [42]:
structure_test = Sequential()
e = Embedding(100000, 200, input_length=200)
structure_test.add(e)
structure_test.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
structure_test.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 200)          20000000  
_________________________________________________________________
conv1d (Conv1D)              (None, 199, 100)          40100     
Total params: 20,040,100
Trainable params: 20,040,100
Non-trainable params: 0
_________________________________________________________________


Now if we add Global Max Pooling layer, then the pooling layer will extract the maximum value from each filter, and the output dimension will be a just 1-dimensional vector with length as same as the number of filters we applied. This can be directly passed on to a dense layer without flattening.

In [43]:
structure_test = Sequential()
e = Embedding(100000, 200, input_length=200)
structure_test.add(e)
structure_test.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
structure_test.add(GlobalMaxPooling1D())
structure_test.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 200)          20000000  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 199, 100)          40100     
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
Total params: 20,040,100
Trainable params: 20,040,100
Non-trainable params: 0
_________________________________________________________________


Now, let's define a simple CNN going through bigrams on a ticket. The output from global max pooling layer will be fed to a fully connected layer, then finally the output layer. Again I will try three different inputs, static word vectors extracted from Word2Vec, word embedding being learned from scratch with embedding layer, Word2Vec word vectors being updated through training.

In [44]:
model_cnn_01 = Sequential()
e = Embedding(100000, 200, weights=[embedding_matrix], input_length=200, trainable=False)
model_cnn_01.add(e)
model_cnn_01.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_01.add(GlobalMaxPooling1D())
model_cnn_01.add(Dense(256, activation='relu'))
model_cnn_01.add(Dense(74, activation='softmax'))
model_cnn_01.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

Epoch 1/5
781/781 - 7s - loss: 2.0155 - accuracy: 0.4782 - val_loss: 1.2359 - val_accuracy: 0.6634
Epoch 2/5
781/781 - 3s - loss: 0.8579 - accuracy: 0.7569 - val_loss: 0.7455 - val_accuracy: 0.7841
Epoch 3/5
781/781 - 3s - loss: 0.5206 - accuracy: 0.8497 - val_loss: 0.6025 - val_accuracy: 0.8238
Epoch 4/5
781/781 - 3s - loss: 0.3802 - accuracy: 0.8873 - val_loss: 0.5165 - val_accuracy: 0.8527
Epoch 5/5
781/781 - 3s - loss: 0.3030 - accuracy: 0.9072 - val_loss: 0.5120 - val_accuracy: 0.8479


In [46]:
model_cnn_02 = Sequential()
e = Embedding(100000, 200, input_length=200)
model_cnn_02.add(e)
model_cnn_02.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_02.add(GlobalMaxPooling1D())
model_cnn_02.add(Dense(256, activation='relu'))
model_cnn_02.add(Dense(74, activation='softmax'))
model_cnn_02.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_02.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

Epoch 1/5
781/781 - 91s - loss: 2.1598 - accuracy: 0.4558 - val_loss: 1.1445 - val_accuracy: 0.6915
Epoch 2/5
781/781 - 90s - loss: 0.6566 - accuracy: 0.8241 - val_loss: 0.5242 - val_accuracy: 0.8575
Epoch 3/5
781/781 - 90s - loss: 0.2899 - accuracy: 0.9179 - val_loss: 0.4128 - val_accuracy: 0.8831
Epoch 4/5
781/781 - 90s - loss: 0.1916 - accuracy: 0.9437 - val_loss: 0.3879 - val_accuracy: 0.8905
Epoch 5/5
781/781 - 90s - loss: 0.1595 - accuracy: 0.9527 - val_loss: 0.3820 - val_accuracy: 0.8997


In [47]:
model_cnn_03 = Sequential()
e = Embedding(100000, 200, weights=[embedding_matrix], input_length=200, trainable=True)
model_cnn_03.add(e)
model_cnn_03.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_03.add(GlobalMaxPooling1D())
model_cnn_03.add(Dense(256, activation='relu'))
model_cnn_03.add(Dense(74, activation='softmax'))
model_cnn_03.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_03.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

Epoch 1/5
781/781 - 91s - loss: 1.8500 - accuracy: 0.5227 - val_loss: 1.0158 - val_accuracy: 0.7127
Epoch 2/5
781/781 - 89s - loss: 0.6592 - accuracy: 0.8149 - val_loss: 0.5878 - val_accuracy: 0.8258
Epoch 3/5
781/781 - 89s - loss: 0.3603 - accuracy: 0.8934 - val_loss: 0.4969 - val_accuracy: 0.8568
Epoch 4/5
781/781 - 88s - loss: 0.2684 - accuracy: 0.9198 - val_loss: 0.4683 - val_accuracy: 0.8728
Epoch 5/5
781/781 - 88s - loss: 0.2152 - accuracy: 0.9332 - val_loss: 0.3956 - val_accuracy: 0.8805


In [48]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model

ticket_input = Input(shape=(200,), dtype='int32')

ticket_encoder = Embedding(100000, 200, weights=[embedding_matrix], input_length=200, trainable=True)(ticket_input)
bigram_branch = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(ticket_encoder)
bigram_branch = GlobalMaxPooling1D()(bigram_branch)
trigram_branch = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(ticket_encoder)
trigram_branch = GlobalMaxPooling1D()(trigram_branch)
fourgram_branch = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(ticket_encoder)
fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

merged = Dense(256, activation='relu')(merged)
merged = Dropout(0.2)(merged)
merged = Dense(74)(merged)
output = Activation('softmax')(merged)
model = Model(inputs=[ticket_input], outputs=[output])
model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 200, 200)     20000000    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 199, 100)     40100       embedding_9[0][0]                
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 198, 100)     60100       embedding_9[0][0]                
______________________________________________________________________________________________

In [49]:
from keras.callbacks import ModelCheckpoint

filepath="CNN_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.fit(x_train_seq, y_train, batch_size=32, epochs=5,
                     validation_data=(x_val_seq, y_validation)) #, callbacks = [checkpoint])

Epoch 1/5
781/781 [==============================] - 91s 115ms/step - loss: 2.5075 - accuracy: 0.3919 - val_loss: 0.7200 - val_accuracy: 0.7944
Epoch 2/5
781/781 [==============================] - 89s 115ms/step - loss: 0.6113 - accuracy: 0.8314 - val_loss: 0.3773 - val_accuracy: 0.8901
Epoch 3/5
781/781 [==============================] - 90s 115ms/step - loss: 0.3056 - accuracy: 0.9104 - val_loss: 0.3877 - val_accuracy: 0.8831
Epoch 4/5
781/781 [==============================] - 89s 114ms/step - loss: 0.2449 - accuracy: 0.9255 - val_loss: 0.3417 - val_accuracy: 0.8985
Epoch 5/5
781/781 [==============================] - 90s 115ms/step - loss: 0.1812 - accuracy: 0.9434 - val_loss: 0.3387 - val_accuracy: 0.9023


In [50]:
from keras.models import load_model
#loaded_CNN_model = load_model('CNN_best_weights.02-0.8333.hdf5')
#loaded_CNN_model.evaluate(x=x_val_seq, y=y_validation)
model.evaluate(x=x_val_seq, y=y_validation)

98/98 [==============================] - 0s 4ms/step - loss: 0.3387 - accuracy: 0.9023


[0.3387061357498169, 0.9023061990737915]

## Final Model Evaluation with Test Set

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(max_features=100000,ngram_range=(1, 3))
tvec.fit(x_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0,
                max_features=100000, min_df=1, ngram_range=(1, 3), norm='l2',
                preprocessor=None, smooth_idf=True, stop_words=None,
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [52]:
x_train_tfidf = tvec.transform(x_train)
x_test_tfidf = tvec.transform(x_test)

In [53]:
sequences_test = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(sequences_test, maxlen=200)

In [54]:
model.evaluate(x=x_test_seq, y=y_test)

98/98 [==============================] - 0s 4ms/step - loss: 0.3215 - accuracy: 0.8985


[0.3215217590332031, 0.8984950184822083]

In [55]:
yhat_cnn = model.predict(x_test_seq)

In [56]:
# A method to train and test the model
def run_dl_model(model, X_train, X_test, y_train, y_test, 
                       epochs=10,batch_size=128, logs_dir="my_logs"):

    tf.keras.backend.clear_session()
    #checkpoint_every_epoch = get_checkpoint_every_epoch()
    run_logdir = get_run_logdir(root_logdir=logs_dir) # e.g., './my_logs/' + run_id
    #tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
    #checkpoint_best_only = get_checkpoint_best_only()
    early_stopping = get_early_stopping(monitor='val_loss', mode='min',patience=5)
    reduce_lr = reduce_learning_rate_on_plateaue(monitor='val_loss', mode='min', patience=5)
    with tf.device("/gpu:0"): 
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=epochs, batch_size=batch_size,verbose=1,
                            callbacks=[#tensorboard_cb, PrintValTrainRatioCallback(),
                            #checkpoint_every_epoch,
                            #checkpoint_best_only,
                            reduce_lr ,
                            print_lr,
                            early_stopping
                            ])
      
    return history

In [57]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 3.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=9260c3e890beb88ffa1d344aaa3bf75f453c3b6bf04233ecdf84da3d49dea09f
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=2301ff4c0cf0959c61013f88a3055ea445a351cabc0db0170f3f72066a80d1d2
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [58]:
import os
from kerastuner import RandomSearch
from kerastuner.engine import hyperparameters
import kerastuner as kt

In [59]:
def get_evaluation_accuracy(model, X_test, y_test):
    """Test model classification accuracy"""
    eval_loss, eval_acc = model.evaluate(x=X_test, y=y_test, verbose=0)
    print('Evaluation Accuracy: {acc:0.3f}'.format(acc=eval_acc))
    print('Evaluation Loss: {acc:0.3f}'.format(acc=eval_loss))

# Function to build Deep NN
def build_dnn_model(shape, nClasses, dropout=0.3):
    model = Sequential()
    node = 512 # number of nodes
    nLayers = 4 # number of  hidden layer
    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
    model.add(Dense(nClasses, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    print(model.summary())
    return model

def get_optimizer():
    return tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)

def build_cnn_model(word_index, embeddings_matrix, nclasses,dropout=0.2):
    model = Sequential()
    embedding_layer = Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embeddings_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True)
  # applying a more complex convolutional approach
    convs = []
    filter_sizes = []
    layer = 5
    for fl in range(0,layer):
        filter_sizes.append((fl+2))
    node = 128
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    for fsz in filter_sizes:
        l_conv = Conv1D(node, kernel_size=fsz, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(5)(l_conv)
        #l_pool = Dropout(0.25)(l_pool)
        convs.append(l_pool)
    l_merge = Concatenate(axis=1)(convs)
    l_cov1 = Conv1D(node, 5, activation='relu')(l_merge)
    l_cov1 = Dropout(dropout)(l_cov1)
    l_batch1 = BatchNormalization()(l_cov1)
    l_pool1 = MaxPooling1D(5)(l_batch1)
    l_cov2 = Conv1D(node, 5, activation='relu')(l_pool1)
    l_cov2 = Dropout(dropout)(l_cov2)
    l_batch2 = BatchNormalization()(l_cov2)
    l_pool2 = MaxPooling1D(30)(l_batch2)
    l_flat = Flatten()(l_pool2)
    l_dense = Dense(1024, activation='relu')(l_flat)
    l_dense = Dropout(dropout)(l_dense)
    l_dense = Dense(512, activation='relu')(l_dense)
    l_dense = Dropout(dropout)(l_dense)
    preds = Dense(nclasses, activation='softmax')(l_dense)
    model = Model(sequence_input, preds)
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=get_optimizer(),
                  metrics=['accuracy'])
    
    print(model.summary())
    return model

# Build GRU model
def build_gru_model(word_index, embeddings_matrix, nclasses,dropout=0.2):
    model = Sequential()
    hidden_layer = 6
    gru_node = 32
    
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embeddings_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    print(gru_node)
    for i in range(0,hidden_layer):
        model.add(tf.compat.v1.keras.layers.CuDNNGRU(gru_node,return_sequences=True))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
    model.add(tf.compat.v1.keras.layers.CuDNNGRU(gru_node))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(nclasses, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                      optimizer=get_optimizer(),
                      metrics=['accuracy'])
    
    print(model.summary())
    return model

# Build lstm model

def build_lstm_model(#word_index, 
                     nclasses=74,
                     dropout=0.2,
                     hidden_layer=6,
                     lstm_node = 3,
                     dense_node= 256,
                     loss='adam',
#                     optimizer=get_optimizer(),
                     accuracy='accuracy',
                     activation='relu',
                     output_activation='softmax'
                    ):
    print(embedding_matrix)
    print(len(word_index) + 1)
    print(EMBEDDING_DIM)
    model = Sequential() 
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    #print(gru_node)
    for i in range(0,hidden_layer):
        model.add(tf.compat.v1.keras.layers.CuDNNLSTM(lstm_node,return_sequences=True))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
    model.add(tf.compat.v1.keras.layers.CuDNNLSTM(lstm_node))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(dense_node, activation=activation ))
    model.add(BatchNormalization())
    model.add(Dense(nclasses, activation=output_activation))
    model.compile(loss=loss,
                      optimizer='adam',
                      metrics=[accuracy])
    
    print(model.summary())
    return model

def get_model(input_shape, batch_size, len):
  """
  model = Sequential([
                      Flatten(input_shape=input_shape),
                      Dense(128*6, activation='relu'),
                      Dense(128*6, activation='relu'
                            #kernel_regularizer=tf.keras.regularizers.l2(0.01)
                      ),
                      Dense(128*6, activation='relu'
                            #kernel_regularizer=tf.keras.regularizers.l2(0.01)
                      ),
                      Dense(10, activation='softmax')

  ])
  model = Sequential([
                      Flatten(input_shape=input_shape),
                      #BatchNormalization(),
                      Dense(128*6, kernel_initializer='he_uniform'),
                      #BatchNormalization(),
                      Activation('relu', ),
                      Dropout(0.2),
                      Dense(128*6, kernel_initializer='he_uniform'),
                      #BatchNormalization(),
                      Activation('relu'),
                      Dropout(0.2),
                      Dense(128*6, kernel_initializer='he_uniform'),
                      #BatchNormalization(),
                      Activation('relu'),
                      Dense(10, activation='softmax')

  ])
  """
  #for SELU - prerequisites are initialzer=lecun_normal, input should be standardized, 
  
  model = Sequential([
                      Flatten(input_shape=input_shape),
                      BatchNormalization(),
                      Dense(128*6, kernel_initializer='lecun_normal'),
                      BatchNormalization(),
                      Activation('selu', ),
                      Dropout(0.2),
                      Dense(128*6, kernel_initializer='lecun_normal'),
                      BatchNormalization(),
                      Activation('selu'),
                      Dropout(0.2),
                      Dense(128*6, kernel_initializer='lecun_normal'),
                      BatchNormalization(),
                      Activation('selu'),
                      Dense(10, activation='softmax')

  ])
  #s = 20 * len // batch_size # number of steps in 20 epochs (batch size = 32)
  #learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
  #optimizer = tf.keras.optimizers.Adam(lr=1e-4, amsgrad=True)
  optimizer=tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True) 
  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  return model



def reduce_learning_rate_on_plateaue(patience=3,monitor='val_loss',mode='min'):
    #reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor=monitor, factor=0.3, patience=patience,
    #                                                 mode=mode, verbose=1)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor=monitor, 
                                  factor=0.1, 
                                  patience=10, 
                                  verbose=1, 
                                  mode=mode, 
                                  min_delta=0.0001, 
                                  cooldown=5, 
                                  min_lr=1e-6)
    return reduce_lr

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
              lambda epoch: 1e-3 * 10**(epoch / 10))


def get_early_stopping(patience=3,monitor='val_loss',mode='min'):
    """
    This function should return an EarlyStopping callback that stops training when
    the validation (testing) accuracy has not improved in the last 3 epochs.
    HINT: use the EarlyStopping callback with the correct 'monitor' and 'patience'
    """
    return tf.keras.callbacks.EarlyStopping(
        monitor=monitor,  patience=patience, verbose=1, mode=mode
    )   

def get_checkpoint_every_epoch( checkpoint_path = 'checkpoints_every_epoch/checkpoint_{epoch:03d}'):
    """
    This function should return a ModelCheckpoint object that:
    - saves the weights only at the end of every epoch
    - saves into a directory called 'checkpoints_every_epoch' inside the current working directory
    - generates filenames in that directory like 'checkpoint_XXX' where
      XXX is the epoch number formatted to have three digits, e.g. 001, 002, 003, etc.
    """
    checkpoint_path = checkpoint_path
    return ModelCheckpoint(filepath=checkpoint_path,
                             save_freq='epoch', #if save_freq is an integer like 1000, save every 1000 samples 
                             save_weights_only=True,verbose=1)   

def get_checkpoint_best_only(monitor='val_accuracy',
                             checkpoint_best_path = 'checkpoints_best_only/checkpoint'):
    """
    This function should return a ModelCheckpoint object that:
    - saves only the weights that generate the highest validation (testing) accuracy
    - saves into a directory called 'checkpoints_best_only' inside the current working directory
    - generates a file called 'checkpoints_best_only/checkpoint' 
    """
    checkpoint_best_path = checkpoint_best_path
    return ModelCheckpoint(filepath=checkpoint_best_path,
                                  save_weights_only=True,
                                  save_freq='epoch',
                                  save_best_only=False,
                                  verbose=1,
                                  monitor=monitor)    
    
def get_run_logdir(root_logdir="my_logs"):
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

#run_logdir = get_run_logdir() # e.g., './my_logs/' + run_id

#tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

class LossAndMetricCallback(tf.keras.callbacks.Callback):

    # Print the loss after every second batch in the training set
    def on_train_batch_end(self, batch, logs=None):
        if batch %2 ==0:
            print('\n After batch {}, the loss is {:7.2f}.'.format(batch, logs['loss']))
    
    # Print the loss after each batch in the test set
    def on_test_batch_end(self, batch, logs=None):
        print('\n After batch {}, the loss is {:7.2f}.'.format(batch, logs['loss']))

    # Print the loss and mean absolute error after each epoch
    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {}: Average loss is {:7.2f}, mean absolute error is {:7.2f}.'.format(epoch, logs['loss'], logs['mae']))
    
    # Notify the user when prediction has finished on each batch
    def on_predict_batch_end(self,batch, logs=None):
        print("Finished prediction on batch {}!".format(batch))

class PrintValTrainRatioCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

class lr_print_cb(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.lr
        #decay = self.model.optimizer.decay
        #iterations = self.model.optimizer.iterations
        #lr_with_decay = lr / (1. + decay * K.cast(iterations, K.dtype(decay)))
        print("Learning Rate = ", lr)
        
print_lr = lr_print_cb()

test_logdir = get_run_logdir()
writer = tf.summary.create_file_writer(test_logdir)
with writer.as_default():
    for step in range(1, 1000 + 1):
        tf.summary.scalar("my_scalar", np.sin(step / 10), step=step)
        data = (np.random.randn(100) + 2) * step / 100 # some random data
        tf.summary.histogram("my_hist", data, buckets=50, step=step)
        images = np.random.rand(2, 32, 32, 3) # random 32×32 RGB images
        tf.summary.image("my_images", images * step / 1000, step=step)
        texts = ["The step is " + str(step), "Its square is " + str(step**2)]
        tf.summary.text("my_text", texts, step=step)
        sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio("my_audio", audio, sample_rate=48000, step=step)
        
#Function to plot accuracy
def plot_accuracy(history):
  try:
      plt.plot(history.history['accuracy'])
      plt.plot(history.history['val_accuracy'])
  except KeyError:
      plt.plot(history.history['acc'])
      plt.plot(history.history['val_acc'])
  plt.title('Accuracy vs. epochs')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Training', 'Validation'], loc='lower right')
  plt.show() 

#Function to plot loss
def plot_loss(history, scale=1):
  try:
      plt.plot(history.history['loss'])
      plt.plot( [x / scale for x in history.history['val_loss']] )
  except KeyError:
      plt.plot(history.history['loss'])
      plt.plot([x / scale for x in history.history['val_loss']] )
  plt.title('Loss vs. epochs')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Training', 'Validation'], loc='upper right')
  plt.show() 

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

#Function to plot the confusion matrix that will be called at the end of each epoch
#The confusion matrix will be visible in the Tensor Board  
class_names=['0','1','2','3','4','5','6','7','8','9']
def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Normalize the confusion matrix.
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure

def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred_raw = model1.predict(test_images)
  test_pred = np.argmax(test_pred_raw, axis=1)

  # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=class_names)
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("Confusion Matrix", cm_image, step=epoch)

# Define the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

def print_values(history):
  print("Mean Accuracy for the validation dataset: ")
  print(np.array(history.history['val_accuracy']).mean())
  print("Mean Loss for the validation dataset: ")
  print(np.array(history.history['val_loss']).mean())
  print("Mean Accuracy for the training dataset: ")
  print(np.array(history.history['accuracy']).mean())
  print("Mean Loss for the training dataset: ")
  print(np.array(history.history['loss']).mean())

def print_results(model, X_train, X_test, y_train, y_test):
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_train_pred = model.predict(X_train)
    y_train_pred = np.argmax(y_train_pred, axis=1)
    
    print('Prediction Model:', model)
    print('-'*80)
    print('Training accuracy: %.2f%%' % (accuracy_score(y_train,y_train_pred) * 100))
    print('Testing accuracy: %.2f%%' % (accuracy_score(y_test, y_pred) * 100))
    print('-'*80)
    #print('Confusion matrix:\n %s' % (confusion_matrix(y_test, y_pred)))
    cm = confusion_matrix(y_test, y_pred)
    
    
    cm_df = pd.DataFrame(cm)
    plt.figure(figsize=(20,10))  
    sns.heatmap(cm_df, annot=True)
    print('-'*80)
    print('Classification report:\n %s' % (classification_report(y_test, y_pred)))

def print_scores(model,X_test,y_test):
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    precision = precision_score(y_test, y_pred, average='weighted')
    print('Precision: %f' % precision)
    recall = recall_score(y_test, y_pred, average='weighted')
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print('F1 score: %f' % f1)
    print('-'*80)

    
def print_classification_report(model,X_test,y_test):
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    print('Classification report:\n %s' % (classification_report(y_test, y_pred)))

    
def print_confusion_matrix(model,X_test,y_test):
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    cm = confusion_matrix(y_test, y_pred)  
    cm_df = pd.DataFrame(cm)
    plt.figure(figsize=(20,10))  
    sns.heatmap(cm_df, annot=True)


In [60]:
#%%script false --no-raise-error
from tensorboard.plugins.hparams import api as hp2
def build_model(hp):


    ticket_input = Input(shape=(200,), dtype='int32')

    ticket_encoder = Embedding(100000, 200, weights=[embedding_matrix], input_length=200, trainable=True)(ticket_input)
    bigram_branch = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(ticket_encoder)
    bigram_branch = GlobalMaxPooling1D()(bigram_branch)
    trigram_branch = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(ticket_encoder)
    trigram_branch = GlobalMaxPooling1D()(trigram_branch)
    fourgram_branch = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(ticket_encoder)
    fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
    merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

    merged = Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=512, step=16),
        activation='relu')(merged)  
    merged = Dropout(
                hp.Choice('dropout_1', values = [0.1,0.2,0.3,0.4,0.5]),
            )(merged)
    merged = Dense(74)(merged)
    output = Activation('softmax')(merged)
    model = Model(inputs=[ticket_input], outputs=[output])
    model.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])
    model.summary()

    return model

    #word_index, embeddings_matrix, nclasses = get_params()
    model = Sequential()
    hidden_layer = 6
    #gru_node = 32
    dropout=0.2
    #HP_DROPOUT = hp2.HParam('dropout', hp2.RealInterval(0.1, 0.2))

    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    #print(gru_node)
    for i in range(0,hidden_layer):
        model.add(tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(
            units=hp.Int('lstm_node_1_units', min_value=16, max_value=256, step=16),
            #units=hp.Choice('lstm_node_1_units', values = [32,128,256]),
           return_sequences=True
        )))
        model.add(
            Dropout(
                hp.Choice('dropout_1', values = [0.1,0.2,0.3,0.4,0.5]),
            ))
        model.add(BatchNormalization())
    model.add(tf.keras.layers.Bidirectional(
        #tf.compat.v1.keras.layers.CuDNNGRU(gru_node)
        tf.compat.v1.keras.layers.CuDNNLSTM(
        units=hp.Int('lstm_node_2_units', min_value=16, max_value=256, step=16)
            #units=hp.Choice('lstm_node_2_units', values = [32,128,256])
    )))
    model.add(
        Dropout(
                hp.Choice('dropout_2', values = [0.1,0.2,0.3,0.4,0.5]),   
        ))
    model.add(BatchNormalization())
    model.add(
        #Dense(256, activation='relu')
        Dense(units=hp.Int('dense_1_units', min_value=32, max_value=512, step=16),
              #units=hp.Choice('dense_1_units', values = [32,128,256]),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dense(74, activation='softmax'))
    #lr = hp.Choice('learning_rate', values=[0.001, 1e-2, 1e-3, 1e-4])
    #momentum = hp.Choice('momentum', values=[0.0, 0.2, 0.4, 0.6, 0.8, 0.9])
    optimizer=tf.keras.optimizers.SGD(hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]),
                                      hp.Choice('momentum', values=[0.0, 0.2, 0.4, 0.6, 0.8, 0.9]))
    model.compile(optimizer=optimizer,
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy']
                )
    return model

In [61]:
#%%script false --no-raise-error
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials=10, 
                            directory='outputs', 
                            project_name='TicketAssignment'
                            )

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 200)     20000000    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 199, 100)     40100       embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 198, 100)     60100       embedding[0][0]                  
______________________________________________________________________________________________

In [62]:
#%%script false --no-raise-error
#logs_dir="logs_model_CNN"
#run_logdir = get_run_logdir(root_logdir=logs_dir) # e.g., './my_logs/' + run_id
#tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
checkpoint_best_only = get_checkpoint_best_only()
early_stopping = get_early_stopping(monitor='val_loss', mode='min',patience=10)
reduce_lr = reduce_learning_rate_on_plateaue(monitor='val_loss', mode='min', patience=10)


tuner_search.search(x_train_seq, y_train,epochs=10, batch_size=16, 
                   validation_data=(x_val_seq, y_validation), 
                   verbose=1,
                   callbacks=[#tensorboard_cb,
                            #PrintValTrainRatioCallback(),
                            #checkpoint_every_epoch,
                            #checkpoint_best_only,
                            print_lr,
                            reduce_lr , early_stopping
                            ])

#history = run_dl_model(model_GRU, X_train_Glove, X_test_Glove, y_train, y_test,
#                   epochs=3, batch_size=32,logs_dir="logs_model_CNN")#    with tf.device("/gpu:0"): 
#        history = model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=epochs, batch_size=batch_size,verbose=1,
#                            callbacks=[tensorboard_cb, PrintValTrainRatioCallback(),
#                            #checkpoint_every_epoch,
#                            checkpoint_best_only,
#                            reduce_lr , early_stopping
#                            ])

Trial 10 Complete [00h 27m 58s]
val_accuracy: 0.9061499238014221

Best val_accuracy So Far: 0.9167200326919556
Total elapsed time: 04h 49m 52s
INFO:tensorflow:Oracle triggered exit


In [63]:
#while True:pass

In [64]:
 #%%script false --no-raise-error
hyperparams = tuner_search.get_best_hyperparameters(num_trials = 5)[0]
hyperparams

In [65]:
#%%script false --no-raise-error
best_model = tuner_search.get_best_models(num_models=1)[0]
loss, mse = best_model.evaluate(x_val_seq, y_validation)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 200)     20000000    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 199, 100)     40100       embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 198, 100)     60100       embedding[0][0]                  
______________________________________________________________________________________________

In [66]:
#%%script false --no-raise-error
history = run_dl_model(best_model, x_train_seq, x_val_seq, y_train,  
                   y_validation, epochs=100,batch_size=16,logs_dir="logs_model_CNN")#  

  #                 run_dl_model(model, X_train, X_test, y_train, y_test, 
   #                    epochs=10,batch_size=128, logs_dir="my_logs")

Epoch 1/100
1561/1561 [==============================] - 169s 108ms/step - loss: 0.2487 - accuracy: 0.9199 - val_loss: 0.3775 - val_accuracy: 0.9061
Learning Rate =  <tf.Variable 'Adam/learning_rate:0' shape=() dtype=float32, numpy=0.001>
Epoch 2/100
1561/1561 [==============================] - 168s 107ms/step - loss: 0.2547 - accuracy: 0.9234 - val_loss: 0.4267 - val_accuracy: 0.9017
Learning Rate =  <tf.Variable 'Adam/learning_rate:0' shape=() dtype=float32, numpy=0.001>
Epoch 3/100
1561/1561 [==============================] - 168s 108ms/step - loss: 0.2379 - accuracy: 0.9281 - val_loss: 0.4342 - val_accuracy: 0.9081
Learning Rate =  <tf.Variable 'Adam/learning_rate:0' shape=() dtype=float32, numpy=0.001>
Epoch 4/100
1561/1561 [==============================] - 168s 108ms/step - loss: 0.2356 - accuracy: 0.9278 - val_loss: 0.4028 - val_accuracy: 0.9116
Learning Rate =  <tf.Variable 'Adam/learning_rate:0' shape=() dtype=float32, numpy=0.001>
Epoch 5/100
1561/1561 [=====================

In [ ]:
while True:pass